In [1]:
%pip install datasets transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
import json

seed = 42

np.random.seed(seed)
random.seed(seed)

Getting the dataset.

In [ ]:
from datasets import load_dataset
sem_eval_2018_task_1 = load_dataset('sem_eval_2018_task_1', 'subtask5.english', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

For each of the files with generated GPT data, we parse the JSON file, drop the missing values and, if necessary, convert the label columns to boolean values. The data is combined with half of the gold standard data. Additionally, the distribution of labels can be verified.

In [ ]:
generated_df = pd.read_csv('generated_tweets_no_role.csv')

In [ ]:
# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data.drop(columns=["Tweet"]).sum()

anger           104
anticipation    723
disgust         158
fear            256
joy             600
love            399
optimism        610
pessimism       495
sadness         296
surprise        387
trust           608
dtype: int64

In [ ]:
# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data.csv', index=False)

In [ ]:
generated_df = pd.read_csv('generated_tweets_no_role_news.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_no_role_news.csv', index=False)

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_1.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_1.csv', index=False)

In [ ]:
generated_data.dtypes

Tweet           object
anger             bool
anticipation      bool
disgust           bool
fear              bool
joy               bool
love              bool
optimism          bool
pessimism         bool
sadness           bool
surprise          bool
trust             bool
dtype: object

In [ ]:
mixed_df.dtypes

ID              object
Tweet           object
anger             bool
anticipation      bool
disgust           bool
fear              bool
joy               bool
love              bool
optimism          bool
pessimism         bool
sadness           bool
surprise          bool
trust             bool
dtype: object

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_2.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_2.csv', index=False)

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_3.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_3.csv', index=False)

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_4.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_4.csv', index=False)

In [ ]:
generated_df = pd.read_csv('generated_tweets_role_5.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('half_gsd_half_gpt_data_role_5.csv', index=False)

## Single-label Data

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_no_role.csv')

# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_single_label_no_role.csv', index=False)

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_role_1.csv')

# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_single_label_role_1.csv', index=False)

In [ ]:
generated_df = pd.read_csv('generated_tweets_single_label_role_1_news.csv')

# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_single_label_role_1_news.csv', index=False)

## Increased size data

In [ ]:
# Load original dataset
df = pd.DataFrame(sem_eval_2018_task_1['train'])

# Select half of the original DataFrame and shuffle
df_shuffled = df.sample(frac=1, random_state=seed).reset_index(drop=True)
half_index = len(df_shuffled) // 2
half_df = df_shuffled.iloc[:half_index].copy()

In [ ]:
generated_df = pd.read_csv('generated_tweets_increased_multi.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_increased_multi.csv', index=False)

In [ ]:
generated_df = pd.read_csv('generated_tweets_increased_single.csv')

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_df.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Turning the data into boolean values
label_columns = ['anticipation', 'optimism', 'trust', 'joy', 'love', 'anger', 'disgust', 'pessimism', 'sadness', 'fear', 'surprise']
generated_data[label_columns] = generated_data[label_columns].astype(bool)
generated_data.drop(columns=["Tweet"]).sum()

# Concatenate half_df and half_augmented_df to create mixed_df
mixed_df = pd.concat([half_df, generated_data], ignore_index=True)

# Save the mixed dataset to a CSV file
mixed_df.to_csv('gsd_gpt_increased_single.csv', index=False)

## DATA Typicality

Inter-dataset typicality.

We are looking at how similar the tweets from the generated data are to the original gold standard data tweets.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Original df
generated_data = pd.read_csv('generated_tweets_no_role.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.96 with standard deviation of 0.10
Typicality for anticipation: 0.96 with standard deviation of 0.10
Typicality for disgust: 0.96 with standard deviation of 0.11
Typicality for fear: 0.98 with standard deviation of 0.12
Typicality for joy: 1.03 with standard deviation of 0.14
Typicality for love: 1.08 with standard deviation of 0.23
Typicality for optimism: 1.13 with standard deviation of 0.16
Typicality for pessimism: 1.13 with standard deviation of 0.13
Typicality for sadness: 1.12 with standard deviation of 0.15
Typicality for surprise: 0.92 with standard deviation of 0.13
Typicality for trust: 1.08 with standard deviation of 0.13

Overall Typicality: 1.03


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_no_role_news.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 1.00 with standard deviation of 0.08
Typicality for anticipation: 1.01 with standard deviation of 0.09
Typicality for disgust: 1.01 with standard deviation of 0.08
Typicality for fear: 1.03 with standard deviation of 0.08
Typicality for joy: 0.97 with standard deviation of 0.09
Typicality for love: 0.96 with standard deviation of 0.19
Typicality for optimism: 1.09 with standard deviation of 0.12
Typicality for pessimism: 1.10 with standard deviation of 0.10
Typicality for sadness: 1.07 with standard deviation of 0.11
Typicality for surprise: 0.98 with standard deviation of 0.20
Typicality for trust: 1.06 with standard deviation of 0.12

Overall Typicality: 1.02


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_1.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.98 with standard deviation of 0.12
Typicality for anticipation: 0.98 with standard deviation of 0.11
Typicality for disgust: 0.98 with standard deviation of 0.11
Typicality for fear: 1.01 with standard deviation of 0.13
Typicality for joy: 0.99 with standard deviation of 0.12
Typicality for love: 1.01 with standard deviation of 0.21
Typicality for optimism: 1.12 with standard deviation of 0.15
Typicality for pessimism: 1.14 with standard deviation of 0.13
Typicality for sadness: 1.11 with standard deviation of 0.14
Typicality for surprise: 0.91 with standard deviation of 0.12
Typicality for trust: 1.07 with standard deviation of 0.15

Overall Typicality: 1.03


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_2.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.95 with standard deviation of 0.12
Typicality for anticipation: 0.98 with standard deviation of 0.12
Typicality for disgust: 0.95 with standard deviation of 0.12
Typicality for fear: 0.98 with standard deviation of 0.12
Typicality for joy: 1.03 with standard deviation of 0.16
Typicality for love: 1.08 with standard deviation of 0.27
Typicality for optimism: 1.16 with standard deviation of 0.18
Typicality for pessimism: 1.13 with standard deviation of 0.13
Typicality for sadness: 1.10 with standard deviation of 0.14
Typicality for surprise: 0.91 with standard deviation of 0.14
Typicality for trust: 1.09 with standard deviation of 0.15

Overall Typicality: 1.03


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_3.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.95 with standard deviation of 0.11
Typicality for anticipation: 0.98 with standard deviation of 0.11
Typicality for disgust: 0.95 with standard deviation of 0.12
Typicality for fear: 0.99 with standard deviation of 0.13
Typicality for joy: 1.03 with standard deviation of 0.15
Typicality for love: 1.08 with standard deviation of 0.25
Typicality for optimism: 1.16 with standard deviation of 0.17
Typicality for pessimism: 1.14 with standard deviation of 0.15
Typicality for sadness: 1.11 with standard deviation of 0.17
Typicality for surprise: 0.91 with standard deviation of 0.14
Typicality for trust: 1.09 with standard deviation of 0.15

Overall Typicality: 1.03


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_4.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.98 with standard deviation of 0.11
Typicality for anticipation: 0.97 with standard deviation of 0.11
Typicality for disgust: 0.99 with standard deviation of 0.11
Typicality for fear: 1.01 with standard deviation of 0.12
Typicality for joy: 0.96 with standard deviation of 0.10
Typicality for love: 0.97 with standard deviation of 0.18
Typicality for optimism: 1.10 with standard deviation of 0.13
Typicality for pessimism: 1.16 with standard deviation of 0.13
Typicality for sadness: 1.14 with standard deviation of 0.14
Typicality for surprise: 0.91 with standard deviation of 0.16
Typicality for trust: 1.02 with standard deviation of 0.13

Overall Typicality: 1.02


In [ ]:
# Sample data
generated_data = pd.read_csv('generated_tweets_role_5.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)
generated_data = generated_data.dropna()

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 1.00 with standard deviation of 0.09
Typicality for anticipation: 1.01 with standard deviation of 0.09
Typicality for disgust: 1.01 with standard deviation of 0.09
Typicality for fear: 1.03 with standard deviation of 0.09
Typicality for joy: 0.96 with standard deviation of 0.09
Typicality for love: 0.95 with standard deviation of 0.18
Typicality for optimism: 1.08 with standard deviation of 0.13
Typicality for pessimism: 1.11 with standard deviation of 0.11
Typicality for sadness: 1.07 with standard deviation of 0.12
Typicality for surprise: 0.98 with standard deviation of 0.21
Typicality for trust: 1.05 with standard deviation of 0.12

Overall Typicality: 1.02


Single-label

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Original df
generated_data = pd.read_csv('generated_tweets_single_label_no_role.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.95 with standard deviation of 0.11
Typicality for anticipation: 0.97 with standard deviation of 0.11
Typicality for disgust: 0.96 with standard deviation of 0.10
Typicality for fear: 0.98 with standard deviation of 0.11
Typicality for joy: 1.03 with standard deviation of 0.14
Typicality for love: 1.07 with standard deviation of 0.23
Typicality for optimism: 1.13 with standard deviation of 0.17
Typicality for pessimism: 1.14 with standard deviation of 0.17
Typicality for sadness: 1.13 with standard deviation of 0.19
Typicality for surprise: 0.91 with standard deviation of 0.12
Typicality for trust: 1.08 with standard deviation of 0.13

Overall Typicality: 1.03


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Original df
generated_data = pd.read_csv('generated_tweets_single_label_role_1.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 0.99 with standard deviation of 0.12
Typicality for anticipation: 0.98 with standard deviation of 0.12
Typicality for disgust: 0.99 with standard deviation of 0.12
Typicality for fear: 0.99 with standard deviation of 0.12
Typicality for joy: 0.99 with standard deviation of 0.12
Typicality for love: 1.01 with standard deviation of 0.21
Typicality for optimism: 1.11 with standard deviation of 0.15
Typicality for pessimism: 1.12 with standard deviation of 0.12
Typicality for sadness: 1.10 with standard deviation of 0.14
Typicality for surprise: 0.92 with standard deviation of 0.12
Typicality for trust: 1.07 with standard deviation of 0.14

Overall Typicality: 1.03


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data
df = pd.DataFrame(sem_eval_2018_task_1['train'])  # Original df
generated_data = pd.read_csv('generated_tweets_single_label_role_1_news.csv')  # Generated data

# Convert the JSON strings in the single column to dictionaries
parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]

# Create a new DataFrame from the parsed data
generated_data = pd.DataFrame(parsed_data)

# Vectorize the tweets
vectorizer = TfidfVectorizer()
original_vectors = vectorizer.fit_transform(df['Tweet'])
generated_vectors = vectorizer.transform(generated_data['Tweet'])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(generated_vectors, original_vectors)

def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = df.drop(columns=['Tweet', 'ID']).to_numpy()

# Calculate typicality for each emotion
typicality_scores = {}
for i, emotion in enumerate(df.columns[2:]):  # Skipping 'Tweet' and 'ID'
    typicality_scores[emotion] = calculate_typicality(similarity_matrix, labels, i)

# Calculate average typicality for the entire dataset for each emotion
average_typicality = {emotion: np.mean(scores) for emotion, scores in typicality_scores.items()}
std_deviation_typicality = {emotion: np.std(scores) for emotion, scores in typicality_scores.items()}

# Print results with formatting
for emotion in average_typicality:
    avg = average_typicality[emotion]
    std_dev = std_deviation_typicality[emotion]
    print(f"Typicality for {emotion}: {avg:.2f} with standard deviation of {std_dev:.2f}")

# Combine all typicality scores across all emotions
all_typicality_scores = np.concatenate(list(typicality_scores.values()))

# Print overall results
print(f"\nOverall Typicality: {np.mean(all_typicality_scores):.2f}")

Typicality for anger: 1.00 with standard deviation of 0.09
Typicality for anticipation: 1.01 with standard deviation of 0.10
Typicality for disgust: 1.01 with standard deviation of 0.10
Typicality for fear: 1.02 with standard deviation of 0.09
Typicality for joy: 0.97 with standard deviation of 0.10
Typicality for love: 0.95 with standard deviation of 0.20
Typicality for optimism: 1.08 with standard deviation of 0.13
Typicality for pessimism: 1.10 with standard deviation of 0.13
Typicality for sadness: 1.07 with standard deviation of 0.16
Typicality for surprise: 0.98 with standard deviation of 0.20
Typicality for trust: 1.05 with standard deviation of 0.14

Overall Typicality: 1.02


Additionally, we compare the typocality of labels in the first generated dataset, as well as comparing generated datasets to each other.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import json

# Specify the dataset to compare
dataset_name = 'role_1'
filepath = 'generated_tweets_role_1.csv'

# Load the dataset
generated_data = pd.read_csv(filepath)

# Convert JSON strings to dictionaries if needed
if isinstance(generated_data.iloc[0, 0], str) and generated_data.iloc[0, 0].startswith('{'):
    parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]
    generated_data = pd.DataFrame(parsed_data)
    generated_data = generated_data.dropna()

# Extract tweets and emotion labels
tweets = generated_data['Tweet'].tolist()
emotion_labels = generated_data.drop(columns=['Tweet'])  # Adjust if needed

# Fit vectorizer on this dataset's text only
vectorizer = TfidfVectorizer()
vec = vectorizer.fit_transform(tweets)

# Calculate cosine similarity of the dataset with itself
similarity_matrix = cosine_similarity(vec, vec)

# Function to calculate typicality for each emotion
def calculate_typicality(similarities, labels, target_label):
    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Convert labels to numpy array for easier indexing
labels = emotion_labels.to_numpy()

# Dictionary to store typicality scores for each emotion
emotion_typicality_scores = {}

# Calculate typicality for each emotion
for i, emotion in enumerate(emotion_labels.columns):
    typicality_scores = calculate_typicality(similarity_matrix, labels, i)
    emotion_typicality_scores[emotion] = np.mean(typicality_scores)

# Print typicality scores for each emotion
for emotion, score in emotion_typicality_scores.items():
    print(f"Typicality for {emotion} in {dataset_name}: {score:.2f}")

# Print overall typicality score
overall_typicality = np.mean(list(emotion_typicality_scores.values()))
print(f"Overall Typicality for {dataset_name}: {overall_typicality:.2f}")

Typicality for anger in role_1: 1.12
Typicality for anticipation in role_1: 1.56
Typicality for disgust in role_1: 1.01
Typicality for fear in role_1: 1.36
Typicality for joy in role_1: 1.59
Typicality for love in role_1: 1.22
Typicality for optimism in role_1: 1.61
Typicality for pessimism in role_1: 1.66
Typicality for sadness in role_1: 1.69
Typicality for surprise in role_1: 1.24
Typicality for trust in role_1: 1.44
Overall Typicality for role_1: 1.41


Ad

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import json

# List of generated datasets
generated_datasets = {
    'no_role': 'generated_tweets_no_role.csv',
    'no_role_news': 'generated_tweets_no_role_news.csv',
    'role_1': 'generated_tweets_role_1.csv',
    'role_2': 'generated_tweets_role_2.csv',
    'role_3': 'generated_tweets_role_3.csv',
    'role_4': 'generated_tweets_role_4.csv',
    'role_5': 'generated_tweets_role_5.csv',
}

# Load all generated datasets and combine text for vectorizer fitting
all_texts = []
dataset_texts = {}
dataset_labels = {}

for name, filepath in generated_datasets.items():
    generated_data = pd.read_csv(filepath)

    # Convert JSON strings to dictionaries if needed
    if isinstance(generated_data.iloc[0, 0], str) and generated_data.iloc[0, 0].startswith('{'):
        parsed_data = [json.loads(entry) for entry in generated_data.iloc[:, 0]]
        generated_data = pd.DataFrame(parsed_data)
        generated_data = generated_data.dropna()

    tweets = generated_data['Tweet'].tolist()
    labels = generated_data.drop(columns=['Tweet'])  # Emotion labels

    all_texts.extend(tweets)  # Collect all texts for fitting vectorizer
    dataset_texts[name] = tweets  # Store each dataset's texts separately
    dataset_labels[name] = labels.to_numpy()  # Store emotion labels separately

# Fit vectorizer on combined text
vectorizer = TfidfVectorizer()
vectorizer.fit(all_texts)

# Vectorize each dataset using the common vectorizer
vectorized_data = {name: vectorizer.transform(texts) for name, texts in dataset_texts.items()}

# Dictionary to store typicality results between datasets
typicality_results = {}

# Function to calculate typicality for each emotion
def calculate_typicality(similarities, labels, target_label):
    if target_label >= labels.shape[1]:  # Ensure target_label is within bounds
        return np.array([])

    target_indices = np.where(labels[:, target_label] == True)[0]
    non_target_indices = np.where(labels[:, target_label] == False)[0]

    if len(target_indices) == 0 or len(non_target_indices) == 0:
        return np.array([])  # Return empty array if no indices for target or non-target

    sim_target = similarities[:, target_indices]
    sim_non_target = similarities[:, non_target_indices]

    avg_sim_target = np.mean(sim_target, axis=1)
    avg_sim_non_target = np.mean(sim_non_target, axis=1)

    typicality = avg_sim_target / avg_sim_non_target
    return typicality

# Compare each pair of datasets
for name1, vec1 in vectorized_data.items():
    for name2, vec2 in vectorized_data.items():
        if name1 != name2:
            # Align sample sizes
            min_samples = min(vec1.shape[0], vec2.shape[0])
            vec1_aligned = vec1[:min_samples]
            vec2_aligned = vec2[:min_samples]

            # Calculate cosine similarity between the two datasets
            similarity_matrix = cosine_similarity(vec1_aligned, vec2_aligned)

            # Align labels to the smallest number of samples
            labels1 = dataset_labels[name1][:min_samples]
            labels2 = dataset_labels[name2][:min_samples]

            # Calculate typicality scores for each emotion
            num_emotions = min(labels1.shape[1], labels2.shape[1])
            emotion_typicality_scores = {}
            for i in range(num_emotions):  # Iterate over common number of emotions
                typicality_scores = calculate_typicality(similarity_matrix, labels1, i)
                emotion_typicality_scores[f'{name1}_vs_{name2}_{i}'] = np.mean(typicality_scores) if len(typicality_scores) > 0 else np.nan

            # Store the average typicality score for all emotions
            valid_scores = [score for score in emotion_typicality_scores.values() if not np.isnan(score)]
            overall_typicality = np.mean(valid_scores) if len(valid_scores) > 0 else np.nan
            typicality_results[(name1, name2)] = overall_typicality

# Print overall typicality scores between each pair of generated datasets
for (name1, name2), score in typicality_results.items():
    print(f"Typicality between {name1} and {name2}: {score:.2f}")


Typicality between no_role and no_role_news: 1.05
Typicality between no_role and role_1: 1.26
Typicality between no_role and role_2: 1.35
Typicality between no_role and role_3: 1.36
Typicality between no_role and role_4: 1.13
Typicality between no_role and role_5: 1.05
Typicality between no_role_news and no_role: 1.08
Typicality between no_role_news and role_1: 1.06
Typicality between no_role_news and role_2: 1.07
Typicality between no_role_news and role_3: 1.08
Typicality between no_role_news and role_4: 1.05
Typicality between no_role_news and role_5: 1.05
Typicality between role_1 and no_role: 1.23
Typicality between role_1 and no_role_news: 1.04
Typicality between role_1 and role_2: 1.35
Typicality between role_1 and role_3: 1.35
Typicality between role_1 and role_4: 1.15
Typicality between role_1 and role_5: 1.05
Typicality between role_2 and no_role: 1.25
Typicality between role_2 and no_role_news: 1.04
Typicality between role_2 and role_1: 1.29
Typicality between role_2 and role